In [1]:
from pymongo import MongoClient
import requests
import pandas as pd

In [2]:
client = MongoClient("mongodb://192.168.1.75:27017/") # povezivanje na bazu podataka

print("Spojeno na MongoDB:", client) # ime klijenta na koji je spojen
print("Dostupne baze podataka:", client.list_database_names()) # koje su sve baze dostupne

db = client["movies"]
print("Dostupne kolekcije u bazi 'movies':", db.list_collection_names())

collection = db["movies"]
print("Broj dokumenata u kolekciji 'movies':", collection.count_documents({}))

Spojeno na MongoDB: MongoClient(host=['192.168.1.75:27017'], document_class=dict, tz_aware=False, connect=True)
Dostupne baze podataka: ['admin', 'config', 'local', 'movies']
Dostupne kolekcije u bazi 'movies': ['users', 'movies']
Broj dokumenata u kolekciji 'movies': 7667


In [3]:
API_KEY = "caf9acb37c7d917f44ef71f4ad294567"
BASE_URL = "https://api.themoviedb.org/3/search/movie"
BASE_URL_DETAILS = "https://api.themoviedb.org/3/movie"


for movie in collection.find({"poster_url": {"$exists": False}}):
    query = movie["name"]
    params = {
        "api_key": API_KEY,
        "query": query
    }
    response = requests.get(BASE_URL, params=params).json()
    # Provjerite je li pronađen film i dodajte poster u bazu
    if response["results"]:
        poster_path = response["results"][0].get("poster_path", "")
        collection.update_one(
            {"_id": movie["_id"]},
            {"$set": {"poster_url": f"https://image.tmdb.org/t/p/w500{poster_path}"}}
        )
print("Podaci obogaćeni slikama!")

Podaci obogaćeni slikama!


In [9]:
for movie in collection.find({"keywords": {"$size": 0}}):
    query = movie["name"]
    print(f"Procesiram film: {query}")
    params = {
        "api_key": API_KEY,
        "query": query
    }
    response = requests.get(BASE_URL, params=params).json()

    keywords = []  # inicijalna prazna lista za ključne riječi

    if response.get("results"):
        movie_id = response["results"][0].get("id", None)
        if movie_id:
            print(f"Film '{query}' pronađen s TMDB ID: {movie_id}")
            keywords_response = requests.get(f"{BASE_URL_DETAILS}/{movie_id}/keywords", params={"api_key": API_KEY}).json() # dohvaćanje ključnih riječi po _id-u 

            keywords = [kw["name"] for kw in keywords_response.get("keywords", [])]

    if not keywords:
        genre = movie.get("genre", "").lower()
        if genre:
            keywords = [genre]
            print(f"Ključne riječi nisu pronađene. Dodan žanr kao ključna riječ za film '{query}': {keywords}")
        else:
            keywords = ["unknown"]
            print(f"Film '{query}' nema ni ključne riječi ni žanr. Dodano 'unknown'.")

    result = collection.update_one(
        {"_id": movie["_id"]},
        {"$set": {"keywords": keywords}}
    )

    if result.modified_count > 0:
        print(f"Ključne riječi za film '{query}' uspješno ažurirane: {keywords}")
    else:
        print(f"Ažuriranje nije uspjelo za film '{query}'.")

print("Proces obogaćivanja podataka završen.")


Procesiram film: Cattle Annie and Little Britches
Film 'Cattle Annie and Little Britches' pronađen s TMDB ID: 167262
Ključne riječi nisu pronađene. Dodan žanr kao ključna riječ za film 'Cattle Annie and Little Britches': ['drama']
Ključne riječi za film 'Cattle Annie and Little Britches' uspješno ažurirane: ['drama']
Procesiram film: The Private Eyes
Film 'The Private Eyes' pronađen s TMDB ID: 74100
Ključne riječi nisu pronađene. Dodan žanr kao ključna riječ za film 'The Private Eyes': ['comedy']
Ključne riječi za film 'The Private Eyes' uspješno ažurirane: ['comedy']
Procesiram film: Bon Voyage, Charlie Brown (and Don't Come Back!!)
Film 'Bon Voyage, Charlie Brown (and Don't Come Back!!)' pronađen s TMDB ID: 31112
Ključne riječi nisu pronađene. Dodan žanr kao ključna riječ za film 'Bon Voyage, Charlie Brown (and Don't Come Back!!)': ['animation']
Ključne riječi za film 'Bon Voyage, Charlie Brown (and Don't Come Back!!)' uspješno ažurirane: ['animation']
Procesiram film: Little Miss Ma

In [39]:
for movie in collection.find({"$or": [{"budget": ""}, {"gross": ""}]}):
    query = movie["name"]
    params = {
        "api_key": API_KEY,
        "query": query
    }
    response = requests.get(BASE_URL, params=params).json()

    if response["results"]:
        movie_id = response["results"][0].get("id", None)  # Dohvati TMDB ID filma
        if movie_id:
            details_response = requests.get(f"{BASE_URL_DETAILS}/{movie_id}", params={"api_key": API_KEY}).json() # detalji filma oreko _id-a

            budget = details_response.get("budget", "")
            gross = details_response.get("revenue", "")

            collection.update_one(
                {"_id": movie["_id"]},
                {"$set": {"budget": budget, "gross": gross}}
            )

print("Podaci o 'budget' i 'gross' su obogaćeni!")

Podaci o 'budget' i 'gross' su obogaćeni!
